In [ ]:
import argparse
import logging
import math
import os
import random
import sys
import time
from copy import deepcopy
from pathlib import Path
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
import cv2 as cv
import argparse
import pafy
from pytube import YouTube

In [ ]:
#from pytube import YouTube
#YouTube('https://www.youtube.com/watch?v=N556Hv6589c').streams.first().download()

#Malik Willis clip: https://www.youtube.com/clip/UgkxCFRXqzhqDLdOysmXpZe17lUhWf-ZN8Km

In [ ]:
BODY_PARTS = {"Head": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
                "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
                "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "Chest": 14,
                "Background": 15 }

POSE_PAIRS = [ ["Head", "Neck"], ["Neck", "RShoulder"], ["RShoulder", "RElbow"],
             ["RElbow", "RWrist"], ["Neck", "LShoulder"], ["LShoulder", "LElbow"],
             ["LElbow", "LWrist"], ["Neck", "Chest"], ["Chest", "RHip"], ["RHip", "RKnee"],
             ["RKnee", "RAnkle"], ["Chest", "LHip"], ["LHip", "LKnee"], ["LKnee", "LAnkle"] ]

In [ ]:
net = cv.dnn.readNetFromTensorflow("graph_opt.pb")  #create custom

In [ ]:
#cap = cv.VideoCapture("bryce_young.gif")
cap = cv.VideoCapture("train/Malik_Willis_trim.mp4")

In [ ]:
inWidth = int(cap.get(cv.CAP_PROP_FRAME_WIDTH)) #368
inHeight = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))  #368
inRate = int(cap.get(cv.CAP_PROP_FPS))

In [ ]:
inWidth

In [ ]:
inHeight

In [ ]:
inRate

In [ ]:
hasFrame,frame = cap.read()

In [ ]:
frame_count = int(cap.get(cv.CAP_PROP_FRAME_COUNT))

In [ ]:
print('Frame count:',frame_count)

In [ ]:
print('Position:',int(cap.get(cv.CAP_PROP_POS_FRAMES)))

In [ ]:
# use the previous location of the body part if the model is wrong
previous_x, previous_y = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

In [ ]:
# final DF of all body parts
data = []

In [ ]:
count = 0
while cv.waitKey(1) < 0:
    hasFrame, frame = cap.read()
    if not hasFrame:
        #cv.waitKey()
        cv.destroyAllWindows()
        break

    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    
    net.setInput(cv.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight), swapRB=True, crop=False))
    out = net.forward()
    out = out[:, :16, :, :]  
    ### original 19 shapes

    assert(len(BODY_PARTS) == out.shape[1])
    
    points = []
    point_df = pd.DataFrame([])
    x_data, y_data = [], []
    
    for i in range(len(BODY_PARTS)):
        # Slice heatmap of corresponging body's part.
        heatMap = out[0, i, :, :]
        # Originally, we try to find all the local maximums. To simplify a sample
        # we just find a global one. However only a single pose at the same time
        # could be detected this way.
        _, conf, _, point = cv.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3]
        y = (frameHeight * point[1]) / out.shape[2]
        fr = frame_count
        # Add a point if it's confidence is higher than threshold.
        points.append((int(x), int(y)) if conf > 0.5 else None)  ## only appending points with confidence accuracy > 20%
        point_df = point_df.append(points)
        
        # create a duplicate point to be appended to a final_df dataset for analysis
        if conf > 0.5:
            x_data.append(x)
            y_data.append(y)
            xy = tuple(np.array([x,y],int))
            cv.circle(frame,xy,5,(25,0,255),5)
        else:
            x_data.append(previous_x[i])
            y_data.append(previous_y[i])
            
    data.append(x_data + y_data)
    previous_x, previous_y = x_data, y_data
    count +=1


    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        assert(partFrom in BODY_PARTS)
        assert(partTo in BODY_PARTS)

        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]

        if points[idFrom] and points[idTo]:
            cv.line(frame, points[idFrom], points[idTo], (255, 74, 0), 3)
            cv.ellipse(frame, points[idFrom], (4, 4), 0, 0, 360, (255, 255, 255), cv.FILLED)
            cv.ellipse(frame, points[idTo], (4, 4), 0, 0, 360, (255, 255, 255), cv.FILLED)
            cv.putText(frame, str(idFrom), points[idFrom], cv.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255),2,cv.LINE_AA)
            cv.putText(frame, str(idTo), points[idTo], cv.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255),2,cv.LINE_AA)

    t, _ = net.getPerfProfile()
    freq = cv.getTickFrequency() / 1000
    cv.putText(frame, '%.2fms' % (t / freq), (10, 20), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))
    cv.imshow('OpenPose using OpenCV', frame)
    
cv.destroyAllWindows()

In [ ]:
#coordinates of points (missing frames)

#pd.options.display.max_rows = 4000
#point_df

In [ ]:
final_df = pd.DataFrame(data)

In [ ]:
final_df